In [1]:
import sys
import os
# 添加项目根目录到Python路径
sys.path.insert(0, os.path.join(os.getcwd(), '..'))

In [ ]:
from core.llm import get_llm
from core.config import LLMConfig, GraphConfig
from utils.graphDB import graphDB
from core.schema import LLMOutput
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 初始化知识库
gdb_cfg = GraphConfig()
gdb = graphDB(gdb_cfg)
gdb.clear_database()

连接成功！


True

In [5]:
# 初始化大模型
cfg = LLMConfig()
llm = get_llm(cfg)
structure_llm = llm.with_structured_output(LLMOutput)

# 读取example.txt文件
with open('example.txt', 'r') as file:
    texts = file.read()

chunks = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=16).split_text(texts)


In [6]:
# 实体识别的prompts
prompt = """
## 任务
请你担任一名专业的知识提取和组织专家，任务是从给定的文本中提取实体关系信息及其描述，用于构建知识图谱
## 输出格式
按照LLMOutput的格式组织。
## 要求
实体类型包括：人物、组织、地点、事件、概念等。
关系类型包括：人物之间的关系、组织之间的关系、地点之间的关系、事件之间的关系、概念之间的关系等。
关系描述中包含关系的强度、方向等信息。
## 注意：
1. 实体描述中是对实体自身的描述，不能包含关系描述。
2. 关系描述中是对实体之间的关系描述，而不是实体的信息。
文本内容：{}
"""


In [7]:
ret = structure_llm.invoke(prompt.format(chunks[0]))

In [8]:
ret.entities[0]

ExtractEntity(entity_name='林墨', entity_type='人物', entity_description='林墨是小说《雨夜迷踪》的女主角，是林夜的妹妹，性格敏感细腻，对哥哥的失踪极度焦虑。她对哥哥的失踪怀有强烈的情感执念，是推动故事发展的核心人物。')

In [10]:
from settings.schema import Entity,Relation,convert2entity,convert2relation

In [11]:
entities = []
relations = []
for i in range(len(ret.entities)):
    entities.append(convert2entity(ret.entities[i],{"text_unit_ids":[i]}))
for i in range(len(ret.relations)):
    relations.append(convert2relation(ret.relations[i],{"text_unit_ids":[i]}))

In [12]:
entities,relations

([Entity(entity_name='林墨', entity_type='人物', entity_description='林墨是小说《雨夜迷踪》的女主角，是林夜的妹妹，性格敏感细腻，对哥哥的失踪极度焦虑。她对哥哥的失踪怀有强烈的情感执念，是推动故事发展的核心人物。', name_embedding=None, description_embedding=None, text_unit_ids=[0]),
  Entity(entity_name='林夜', entity_type='人物', entity_description='林夜是林墨的哥哥，一名画家，三个月前在雨夜神秘失踪。他生前曾多次前往青山精神病院，其画作风格独特，作品中隐含着与双胞胎秘密相关的线索。', name_embedding=None, description_embedding=None, text_unit_ids=[1]),
  Entity(entity_name='陈远', entity_type='人物', entity_description='陈远是市刑警队的法医，三十出头，是林墨的男友。他专业严谨，对案件调查投入极大精力，是林墨在寻找哥哥下落过程中的主要支持者和调查合作者。', name_embedding=None, description_embedding=None, text_unit_ids=[2]),
  Entity(entity_name='王德明', entity_type='人物', entity_description='王德明是退休的老医生，曾为林墨的母亲接生。他年逾八旬，记忆清晰，掌握着关于林墨与林夜双胞胎身份的关键信息，内心充满愧疚。', name_embedding=None, description_embedding=None, text_unit_ids=[3]),
  Entity(entity_name='青山精神病院', entity_type='地点', entity_description='青山精神病院是一座位于城郊的废弃建筑，十年前曾发生严重火灾，造成十余人死亡。建筑现已破败不堪，但内部地下室却意外保存完好，成为关键线索的发现地。', name_embedding=None, description_embedding

In [13]:
gdb.create_entities_batch(entities),gdb.create_relations_batch(relations)

(True, True)